In [5]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os 
import glob
#import astroalign as aa
from astropy.io import fits

import processing as  pr

### Votre notebook executant le traitement des images doit ressembler à ca .....




In [2]:
data_dir = "/n07data/laigle/data_to_transf/"
output_dir = "/n07data/laigle/M83_PROCESSED/"

## 1. Calibration du master bias

#### recuperer la liste des poses individuelles de bias

In [3]:
list_bias_name = glob.glob('chemin/vers/les/poses/individuelles/de/bias/*')

#### generer le master bias et le sauver au format fits

In [ ]:
master_bias = pr.master_bias(list_bias_name, out_dir ='/chemin/pour/sauver/le/',
                             out_name = 'masterbias.fits', overwrite = 1)

## 2. Calibration du master flat rouge

####  recuperer la liste des poses individuelles de flats rouges

In [ ]:
list_flat_red_name = glob.glob('chemin/vers/les/poses/individuelles/de/flat/red/*')

####  nom du fichier contenant le master bias qui a ete generé juste avant

In [ ]:
master_bias_name = '/chemin/pour/sauver/le/masterbias.fits'

#### generer le master flat et le sauver au format fits

In [ ]:
master_flat_red = pr.master_flat(list_flat_red_name, master_bias_name, out_dir ='/chemin/pour/sauver/le/'
                                 , out_name = 'masterflatred.fits', overwrite = 1)

on peut ensuite generer de la meme maniere les master flat en Ha, et eventuellement dans les autres filtres disponibles ...

## 3. Traitement des images dans le filtre rouge

####  recuperer la liste des poses individuelles des images sciences

In [4]:
list_images_red_name = glob.glob('chemin/vers/les/poses/individuelles/sciences/red/*')

####  nom des fichiers contenant les poses maitres de calibration

In [ ]:
master_bias_name = '/chemin/pour/sauver/le/masterbias.fits'
master_flat_name_red = '/chemin/pour/sauver/le/masterflatred.fits'

#### generer l'image finale et la sauvegarder au format fits

In [ ]:
final_image_red = stacking(list_images_red_name, master_bias_name, master_flat_name_red
                           , out_dir = '/chemin/pour/sauver/l/', out_name = 'image_finale_red.fits'
                           , overwrite = 1)

## 4. Mesure du profil radial de la galaxie

In [ ]:
from photutils.centroids import centroid_quadratic

## xpeak and ypeak are the approximate position of the centroid
xycen = centroid_quadratic(final_image_red, xpeak=2040, ypeak=2080,search_boxsize = 100)

## afficher l'image et y superposer le centroid pour verifier qu'il a bien ete determiné.

In [ ]:
## calcul du profil radial

edge_radii = np.arange(400)
rp,pp,errp = pr.RadialProfile(image, xycen, edge_radii)


## tracer le profil radial ainsi determiné. convertir les pixels en unité sur le ciel (arcsec), puis en kpc.
## Comparer à un profil exponentiel : 

plt.plot(rp,pp)
plt.errorbar(rp,pp,yerr = errp)

a = 
b = 
rp, pp2 = pr.ExponentialProfile(rp, a,b,0)

plt.plot(rp, pp2)

a =
b = 
c = 

rp, pp2 = pr.ExponentialProfile(rp, a,b,100)

plt.plot(rp, pp2)
axes = plt.gca()
axes.set_yscale('log')


## 5. Mesure de la PSF des etoiles

#### explorer https://photutils.readthedocs.io/en/stable/index.html et essayer de mesurer la fonction d'etalement du point des etoiles

In [ ]:
from photutils.detection import DAOStarFinder

from astropy.stats import mad_std

bkg_sigma = mad_std(image)
daofind = DAOStarFinder(fwhm=4.0, threshold=3.0 * bkg_sigma)  
sources = daofind(image) 
for col in sources.colnames:  
    sources[col].info.format = '%.8g'  # for consistent table output
print(sources)  